Alternating Layered Shadow Optimization (ALSO) is a Variational Quantum Algorithm (VQA) that makes use of an alternating layered ansatz and classical shadows to reduce the number of calculations in the estimation of expectation values of observables. In particular, we are interested in minimizing functions of the form $f_{\rho, O}(\theta) = \text{tr}(OU(\theta)\rho U(\theta)^\dag)$, where $\rho$ is an input state and $O$ is a $k$-local observable (ideally 1-local).

First, we need to compute the classical shadow of $\rho$. To get a 'snapshot', we perform a random Pauli measurement on each qubit, obtaining a result $|b_j\rangle$ for the $j^{th}$ qubit. The snapshot is $ \displaystyle\hat\rho = \bigotimes_{j = 1}^n \left ( 3U_j^\dag |b_j \rangle\langle b_j | U_j - \mathbb{I} \right)$, where $U_j$ corresponds to the change of basis associated to the associated Pauli measurement. This can be stored efficiently as $n$ $2\times 2$ matrices. The classical shadow is stored as a set of $T$ snapshots, where $T$ is the 'size' of the shadow.

In [2]:
import numpy as np
from numpy.linalg import norm
from qibo import Circuit, gates
from qibo.symbols import I, X, Y, Z
from qibo.optimizers import optimize
import time

""" Returns a classical shadow of size shadow_size of the state generated by circuit, where
each snapshot consists of an array of num_qubits 2 by 2 matrices"""
def calculate_classical_shadow(circuit, shadow_size, num_qubits):

    # Maps the Pauli measurements to the indeces 0, 1, 2 = X, Y, Z
    Pauli = [gates.X, gates.Y, gates.Z]

    # Measurement basis are chosen at random
    measurement_basis = np.random.randint(0, 3, size = (shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    for i in range(shadow_size):
        # In each snapshot, we perform a measurement in a randomly chosen basis
        c = circuit.copy()
        c.add(gates.M(*range(num_qubits), basis = [Pauli[measurement_basis[i][j]] for j in range(num_qubits)]))
        outcomes[i] = c(nshots = 1).samples(binary = True)[0]
    
    # Possible states in matrix form
    states = [np.array([[1, 0],[0, 0]]), np.array([[0, 0], [0, 1]])]
    # Matrices to invert the implicit unitary operations performed when measuring in different basis
    unitaries = [gates.H(0).matrix(), gates.H(0).matrix()@gates.S(0).dagger().matrix(), gates.I(0).matrix()]
    shadow = np.empty((shadow_size, num_qubits), dtype = np.ndarray)

    # Computes the classical shadows for each snapshots
    for i in range(shadow_size):
        for j in range(num_qubits):
            state = states[int(outcomes[i][j])]
            U = unitaries[measurement_basis[i][j]]
            shadow[i][j] = 3*(U.conj().T @ state @ U) - np.eye(2)

    return shadow

Let $\hat\rho_T$ be the average of the $T$ snapshots in the shadow. Given a set of $k$-local observables $O_1, \dots, O_M$, and a big enough shadow, we can ensure $|\text{tr}(O_i\hat\rho) - \text{tr}(O_i\rho)| \leq \varepsilon$ with probability at least $1 - \delta$. A lower bound on the number of snapshots needed to ensure this is $T = \frac{4^{k + 1}}{\varepsilon^2}\cdot \log \left( \frac{2M}{\delta} \right) \max_i \|O_i\|^2_\infty$, but in practice less copies are actually needed. 

In [3]:

"""Returns a bound on the size of a classical shadow such that the estimation of 
the observables yields a maximum error with a certain failure rate"""
def shadow_bound(observables, error = 0.01, failure_rate = 0.01, locality = 1):
    M = np.size(observables, axis = 0)
    infty_norm = max([norm(obs, ord = np.inf) for obs in observables])
    return int((4**(locality + 1))/(error**2)*np.log(2*M/failure_rate)*(infty_norm**2))

The alternating layered ansatz we used is formed by 'S blocks', 2-local circuits that depend on 12 parameters. If $n$ is the number of qubits ans $d$ is the depth of the circuit, we place the S-blocks so that the whole circuit can be written as 
$\displaystyle U(\theta) = \prod_{j = 1}^d \prod_{i = 1}^{n/2} S(\theta_{ij})[2(i - 1) \oplus j, 2(i - 1) \oplus j \oplus 1]$, where $S(\theta_{ij})[k, l]$ represents the S-block parametrized by $\theta_{ij}\in \mathbb{R}^{12}$ acting on qubits $k$ and $l$.

In [4]:
"""Creates a 2-local block for an alternating layered ansatz"""
def S_block(theta):
    c = Circuit(2)
    c.add(gates.RX(0, theta[0]))
    c.add(gates.RY(0, theta[1]))
    c.add(gates.RX(0, theta[2]))
    c.add(gates.RX(1, theta[3]))
    c.add(gates.RY(1, theta[4]))
    c.add(gates.RX(1, theta[5]))
    c.add(gates.CNOT(0, 1))
    c.add(gates.RX(0, theta[6]))
    c.add(gates.RY(0, theta[7]))
    c.add(gates.RX(0, theta[8]))
    c.add(gates.RX(1, theta[9]))
    c.add(gates.RY(1, theta[10]))
    c.add(gates.RX(1, theta[11]))
    c.add(gates.CNOT(1, 0))
    return c

"""Given a set of (num_qubits//2)*depth*12 angles, generates an alternating layered ansatz"""
def alternating_layered_ansatz(depth, num_qubits, theta):
    c = Circuit(num_qubits, density_matrix=True)
    for j in range(depth):
        for i in range(num_qubits//2):
            c.add(S_block(theta[i][j]).on_qubits((2*(i - 1) + j)%num_qubits, (2*(i - 1)+ j + 1)%num_qubits))
    return c



If our observable is of the form $O = \sum_{i = 1}^n O_i$, where $O_i$ are 1-local observables, the function we want to minimize is of the form $f_{\rho, O}(\theta) = \sum_{i = 1}^n\text{tr}(O_iU(\theta)\rho U(\theta)^\dag) = \sum_{i = 1}^n\text{tr}(U(\theta)^\dag O_iU(\theta)\rho)$ due to the cyclic property of the trace. We will estimate $\rho$ using its classical shadow, so we want to compute the observables $
W_{O_i} = U^\dag O_i U$. However, since the observables are 1-local, they don't interact with the gates outside of their 'light cone', so these gates cancel out between the $U$ and $U^\dag$ terms. Thus, we can use the reduced circuits that comprise only the gates that interact with the observables. The reduced circuits always have $2d$ qubits, where $d$ is the depth of the whole circuit. 

In [5]:

"""An infidelity-like 1-local cost function for the state preparation problem"""
def cost(theta, observables, reduced_shadows):

    global num_qubits, depth
    s = 0
    c = alternating_layered_ansatz(depth, num_qubits, theta.reshape(num_qubits//2, depth, 12))

    #For maximum efficency, we exploit the locality of the observables and make use only of the necessary dimensions
    for i in range(num_qubits):
        u = c.light_cone(i)[0].unitary()
        s += (u.conj().T@observables[i]@u@reduced_shadows[i]).trace()
        
    print(abs(1 - s))
    return abs(1 - s)

Let $A_i$ denote the subregister that $O_i$ acts non-trivially on. Since we are using reduced circuits with only $2d$ qubits, we don't need to compute the whole classical shadow, but only the reduced shadows that act on each of the $A_i$ subregisters. Thus, the computation only depends exponentially on $d$ and not on $n$. If $\rho_{ij}$ represents the $2\times 2$ matrix of the $i^{th}$ snapshot acting on the $j^{th}$ qubit, we compute, for each $i$, $\displaystyle \hat\rho_T|_{A_i} = \frac{1}{T} \sum_{i = 1}^T \bigotimes_{j \in A_i} \rho_{ij}$.

In [11]:
#Circuit parameters
num_qubits = 2
depth = 4

#Generation of random alternating layered ansatz and reduced observables
rng = np.random.default_rng()
theta = rng.uniform(0, 2*np.pi, ((num_qubits//2), depth, 12))
c = alternating_layered_ansatz(depth, num_qubits, theta)
A = [c.light_cone(i)[1] for i in range(num_qubits)]
observables = [(I(A[i][i]).full_matrix(len(A[i])) + Z(A[i][i]).full_matrix(len(A[i])))/(2*num_qubits) for i in range(num_qubits)]

#Generation of the classical shadow and reduced shadows
shadow = calculate_classical_shadow(c, shadow_bound(observables), num_qubits)
reduced_shadows = np.zeros(num_qubits, dtype = np.ndarray)
T = np.size(shadow, axis = 0)
for i in range(num_qubits):
    for j in range(T):
        hat_rho = [1/T]
        for k in list(A[i]):
            hat_rho = np.kron(hat_rho, shadow[j][k])
        reduced_shadows[i] = reduced_shadows[i] + hat_rho


In [12]:
#Optimization procedure
initial_theta = np.zeros((num_qubits//2)*depth*12)
best, params, extra = optimize(cost, initial_theta, args = (observables, reduced_shadows), method = 'L-BFGS-B', options = {'maxiter': 100})


print("Minimum value of the cost function reached:", best)
print("Optimal parameters:\n", params)
print(extra)

rho_hat = alternating_layered_ansatz(depth, num_qubits, params.reshape((num_qubits//2, depth, 12))).invert()().state()
rho = c().state()
rho_shadow = 0
for i in range(T):
    rho_snapshot = [1]
    for j in range(num_qubits):
        rho_snapshot = np.kron(rho_snapshot, shadow[i][j])
    rho_shadow += 1/T*rho_snapshot
print("Fidelity between estimate state and real state:", abs((rho@rho_hat).trace()))
print("Fidelity between shadow state and real state:", abs((rho_shadow@rho).trace()))
print("Fidelity between shadow state and estimate state:", abs(rho_shadow@rho_hat).trace())

0.24322784968634403
0.2432278510531667
0.2432278511764987
0.2432278510531667
0.2432278506896174
0.2432278493638219
0.2432278506896174
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.2432278506896174
0.24322784850641266
0.2432278506896174
0.2432278510531667
0.24322785101564448
0.2432278510531667
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.2432278506896174
0.24322784850641266
0.2432278506896174
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.2432278510531667
0.2432278511764987
0.2432278510531667
0.2432278506896174
0.2432278493638219
0.2432278506896174
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.2432278506896174
0.24322784850641266
0.2432278506896174
0.2432278510531667
0.24322785101564448
0.2432278510531667
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.2432278506896174
0.24322784850641266
0.2432278506896174
0.24322785065466146
0.24322785021894666
0.24322785065466146
0.560519431258446
0.5605194323411933
0.56051